In [1]:
#creating Wage and Cost Data for county level

In [2]:
import pandas as pd
import numpy as np

In [3]:
CostDF = pd.read_csv('createdassets/CostDF.csv',index_col=0)
WageDF = pd.read_csv('createdassets/WageDF.csv',index_col=0)
CompSci = pd.read_csv('createdassets/ComputerScience.csv',index_col=0)

In [4]:
StateNames = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 
              'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 
              'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
              'Maine', 'Maryland', 'Massachusetts','Michigan', 'Minnesota',
              'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 
              'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina',
              'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 
              'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 
              'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 
              'Wisconsin', 'Wyoming']

In [5]:
CostSum = CostDF.where(CostDF['CostType'] != 'Required annual income after taxes').dropna(subset='CostType')
CostSum = CostSum.where(CostSum['CostType'] != 'Required annual income before taxes').dropna(subset='CostType')

for column in CostSum.columns:
    if column == 'State':
        continue
    elif column == 'County':
        continue
    elif column == 'CostType':
        continue
    #elif column == 'CountyState':
    #    continue
    else:
        CostSum[column] = CostSum[column].apply(lambda x: x.replace(',',''))
        CostSum[column] = CostSum[column].apply(lambda x: float(x[1:]))
CostSum = CostSum.groupby(['County','State']).sum()
CostSum['Cost Average'] = CostSum.mean(axis=1)
CostSum.drop(['1 ADULT - 0', '1 ADULT - 1', '1 ADULT - 2', '1 ADULT - 3',
       '2 ADULTS(1 WORKING) - 0', '2 ADULTS(1 WORKING) - 1',
       '2 ADULTS(1 WORKING) - 2', '2 ADULTS(1 WORKING) - 3',
       '2 ADULTS(BOTH WORKING) - 0', '2 ADULTS(BOTH WORKING) - 1',
       '2 ADULTS(BOTH WORKING) - 2', '2 ADULTS(BOTH WORKING) - 3'],axis=1,inplace=True)
CostSum.reset_index(inplace=True)
CostSum

,County,State,Cost Average
0,Abbeville,South Carolina,69896.000000
1,Acadia,Louisiana,71543.416667
2,Accomack,Virginia,68493.083333
3,Ada,Idaho,77379.166667
4,Adair,Iowa,74072.666667
...,...,...,...
3110,Yuma,Arizona,71004.000000
3111,Yuma,Colorado,72831.666667
3112,Zapata,Texas,65964.750000
3113,Zavala,Texas,64554.166667


In [6]:
WageTarg = WageDF.where(WageDF['WageType'] == 'Living Wage').dropna(subset='WageType')
WageTarg.drop(['WageType'],axis=1,inplace=True)
for column in WageTarg.columns:
    if column == 'State':
        continue
    elif column == 'County':
        continue
    elif column == 'WageType':
        continue
    else:
        WageTarg[column] = WageTarg[column].apply(lambda x: float(x[1:]) * 40 * 52)

WageTarg = WageTarg.groupby(['County','State']).sum()
WageTarg['Living Wage'] = WageTarg.mean(axis=1)       
WageTarg.drop(['1 ADULT - 0', '1 ADULT - 1', '1 ADULT - 2', '1 ADULT - 3',
       '2 ADULTS(1 WORKING) - 0', '2 ADULTS(1 WORKING) - 1',
       '2 ADULTS(1 WORKING) - 2', '2 ADULTS(1 WORKING) - 3',
       '2 ADULTS(BOTH WORKING) - 0', '2 ADULTS(BOTH WORKING) - 1',
       '2 ADULTS(BOTH WORKING) - 2', '2 ADULTS(BOTH WORKING) - 3'],axis=1,inplace=True)
WageTarg.reset_index(inplace=True)
WageTarg

,County,State,Living Wage
0,Abbeville,South Carolina,57278.000000
1,Acadia,Louisiana,58480.933333
2,Accomack,Virginia,56080.266667
3,Ada,Idaho,63237.200000
4,Adair,Iowa,60554.000000
...,...,...,...
3110,Yuma,Arizona,58214.000000
3111,Yuma,Colorado,59498.400000
3112,Zapata,Texas,53960.400000
3113,Zavala,Texas,52875.333333


In [7]:
WageTarg['CountyState'] = WageTarg['County'] + WageTarg['State']
CostSum['CountyState'] = CostSum['County'] + CostSum['State']
WageTarg.set_index('CountyState',inplace=True)
CostSum.set_index('CountyState',inplace=True)
CostSum.drop(['County','State'],axis=1,inplace=True)
WageTarg = WageTarg.join(CostSum)
#WageTarg
#CostSum

In [10]:
WageTarg.reset_index(inplace=True)
WageTarg.drop(['CountyState'],axis=1,inplace=True)
WageTarg

,County,State,Living Wage,Cost Average
0,Abbeville,South Carolina,57278.000000,69896.000000
1,Acadia,Louisiana,58480.933333,71543.416667
2,Accomack,Virginia,56080.266667,68493.083333
3,Ada,Idaho,63237.200000,77379.166667
4,Adair,Iowa,60554.000000,74072.666667
...,...,...,...,...
3110,Yuma,Arizona,58214.000000,71004.000000
3111,Yuma,Colorado,59498.400000,72831.666667
3112,Zapata,Texas,53960.400000,65964.750000
3113,Zavala,Texas,52875.333333,64554.166667


In [27]:
def state(x):
    return CompSci.where(CompSci['State'] == x).dropna(subset='State')['Typical Annual Salary'].item()

WageTarg['CompSciSalary'] = [state(x) for x in WageTarg['State']]
WageTarg

,County,State,Living Wage,Cost Average,CompSciSalary
0,Abbeville,South Carolina,57278.000000,69896.000000,"$77,329"
1,Acadia,Louisiana,58480.933333,71543.416667,"$72,760"
2,Accomack,Virginia,56080.266667,68493.083333,"$110,039"
3,Ada,Idaho,63237.200000,77379.166667,"$75,349"
4,Adair,Iowa,60554.000000,74072.666667,"$85,033"
...,...,...,...,...,...
3110,Yuma,Arizona,58214.000000,71004.000000,"$89,591"
3111,Yuma,Colorado,59498.400000,72831.666667,"$103,469"
3112,Zapata,Texas,53960.400000,65964.750000,"$96,439"
3113,Zavala,Texas,52875.333333,64554.166667,"$96,439"


In [28]:
WageTarg.to_csv('CountyComp.csv')